In [1]:
!pip install geemap

In [2]:
!pip install geopandas

In [3]:
import ee

In [4]:
ee.Authenticate()

Enter verification code:  4/1AQlEd8z9NMYMUFNKcRzDAOPlgigEc2N5pM2pTqRta2CnS5XoxRaZcO287zM



Successfully saved authorization token.


In [14]:
import geemap

# Set shapefile path (Update this path as per your file location)
shapefile_path = r'C:\Users\owhor\Downloads\PLUS_softwaredev_cloudsort-main\PLUS_softwaredev_cloudsort-main\Sample_data\your_shapefile.shp'

# Time frame
start_date = "2020-01-01"
end_date = "2021-01-01"

# Tile (Optional)
tile = "33UUP"

# Display map
Map

Map(bottom=2916518.0, center=[47.993770664329645, 12.879045009613039], controls=(WidgetControl(options=['posit…

In [16]:
# Create an interactive map
Map_aoi = geemap.Map()

# Convert shapefile to Earth Engine object
aoi = geemap.shp_to_ee(shapefile_path)

# Add AOI to the map and display
Map_aoi.addLayer(aoi, {}, "AOI")
Map_aoi.centerObject(aoi, 11)
Map_aoi



Map(center=[47.92303544098585, 12.945715130262455], controls=(WidgetControl(options=['position', 'transparent_…

In [17]:
# Load the Sentinel-2 collection and filter by date, bounds, and tile
col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterDate(start_date, end_date)\
    .filterBounds(aoi)\
    .filter(ee.Filter.eq("MGRS_TILE", tile))\
    .sort("CLOUDY_PIXEL_PERCENTAGE")\
    .map(lambda image: image.clip(aoi))


In [19]:
# Calculate the area of the AOI
aoi_geometry = aoi.geometry()
aoi_m2 = aoi_geometry.area(1).getInfo()  # In square meters
aoi_km2 = aoi_m2 / 1e6  # Convert to square kilometers

# Maximum allowed cloud percentage
s2_image_km2 = 10000  # Sentinel-2 image covers 100x100 km
max_cloud_percentage = 100 - ((aoi_km2 * 100) / s2_image_km2)
print("Max allowed cloud percentage:", max_cloud_percentage)


Max allowed cloud percentage: 99.81867953656125


In [20]:
# Filter the collection to include only images with clouds below the threshold
col_sort = col.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", max_cloud_percentage))

# Show the sorted collection
col_sort


In [21]:
# Visualize the first 5 cloud-free images
Map_QA60 = geemap.Map()

# Select number of images to visualize
num_img = 5
images_list = col_sort.toList(num_img)

# Add the images to the map
for i in range(num_img):
    image = ee.Image(images_list.get(i))
    date = image.date().format('yyyy_MM_dd').getInfo()
    
    # Visualization parameters
    vis_params = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.4}
    Map_QA60.addLayer(image, vis_params, f"i{i+1}_{date}_QA60")

Map_QA60.centerObject(aoi, 11)
Map_QA60


Map(center=[47.92303544098585, 12.945715130262455], controls=(WidgetControl(options=['position', 'transparent_…